<a href="https://colab.research.google.com/github/Noahbisht0/Tensorflow2-keras/blob/main/Tensorflow2_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -q tensorflow-datasets

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds 
imdb,info = tfds.load("imdb_reviews",with_info=True,as_supervised = True)

In [ ]:
import numpy as np 
train_data, test_data = imdb['train'], imdb['test']
training_sentences = []
training_labels = []
test_sentences = []
test_labels = []
for s,l in train_data:
  training_sentences.append(s.numpy().decode())
  training_labels.append(l.numpy())
for s,l in test_data:  
  test_sentences.append(s.numpy().decode())
  test_labels.append(l.numpy())
training_labels_final = np.array(training_labels)
test_labels_final =  np.array(test_labels)


In [ ]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)
print(padded[3])
testing_sequences = tokenizer.texts_to_sequences(test_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)


[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0   12    7    2  241    5   20   16    4 8776 2707 2653
   52    2  358    5    2  180   68  138 1401   17   92  203  968   15
   23    1   81    4  192 3109 3037    3    1   16    4  376    5  632
  387  352   37 6355    3 5559 1928   15  208 8596 3433    2  112  365
   48   24   55    1    6 1657   55 1818 4312   41    4 2263    3 1908
    1  141  160  780  111   31   92  116    4  221   20    9  173  279
    3   29   13 1012    2 2814   10 1985]


In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i,'?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])  

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [ ]:
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(6,activation='relu'),
                             tf.keras.layers.Dense(1,activation='sigmoid')
])
model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epochs = 10
model.fit(padded,training_labels_final,epochs=num_epochs,validation_data=(testing_padded,test_labels_final))

Epoch 1/10
782/782 [==============================] - 5s 6ms/step - loss: 0.5958 - accuracy: 0.6485 - val_loss: 0.3517 - val_accuracy: 0.8444
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2488 - accuracy: 0.9054 - val_loss: 0.3593 - val_accuracy: 0.8458
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1125 - accuracy: 0.9711 - val_loss: 0.4439 - val_accuracy: 0.8294
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0299 - accuracy: 0.9968 - val_loss: 0.5210 - val_accuracy: 0.8267
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0082 - accuracy: 0.9994 - val_loss: 0.5930 - val_accuracy: 0.8272
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.6380 - val_accuracy: 0.8285
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.6845 - val_accuracy: 0.8310
Epoch 

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [ ]:
import io 
out_v = io.open('veca.tsv','w',encoding='utf-8')
out_m = io.open('meta.tsv','w',encoding='utf-8')
for word_num in range(1,vocab_size):
  word = reverse_word_index[word_num]
  embedding = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embedding]) + "\n")
out_m.close()
out_v.close()

In [ ]:
try:
    from google.colab import files
except ImportError:
    pass
else:
    files.download('veca.tsv')
    files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>